<a href="https://colab.research.google.com/github/CMPE252/customer-churn-prediction/blob/main/CMPE_252_Customer_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE 252: Customer Churn Prediction

- **Date:** January 22, 2026

- **Data:** [Customer Churn](https://www.kaggle.com/datasets/undersc0re/predict-the-churn-risk-rate)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


# Step 1: Data Cleaning & Pre-processing

## 1.1 Import Data

In [ ]:
# Import the dataset

#------- THIS PART WILL BE DELETED BEFORE SUBMISSION -------
#IF YOU ARE WORKING WITH COPY OF THE FILE WRITE YOUR PATH BELOW
from google.colab import drive
drive.mount('/content/drive')
#path = "/content/drive/MyDrive/Colab/2_ISE 201/Data/Project/"   #Annie
path = "/content/drive/MyDrive/churn.csv"     #Cahide

# Load data
df = pd.read_csv(path, encoding='latin1')
#------- THIS PART WILL BE DELETED BEFORE SUBMISSION ------



#------- THIS LINE SHOULD BE USED IN GITHUB MAKE SURE IT IS WORKING------
#Load data
#df = pd.read_csv("data/churn.csv", encoding='latin1')
#------- THIS LINE SHOULD BE USED IN GITHUB MAKE SURE IT IS WORKING------
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,2,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1
3,3,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,1
4,4,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,1


## 1.2 Data Dimensions & Types

In [ ]:
# Check data size
df.shape

(36992, 24)

There are 36992 rows and 24 columns in the churn dataset.

In [ ]:
# Check columns and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    36992 non-null  int64  
 1   age                           36992 non-null  int64  
 2   gender                        36992 non-null  object 
 3   security_no                   36992 non-null  object 
 4   region_category               31564 non-null  object 
 5   membership_category           36992 non-null  object 
 6   joining_date                  36992 non-null  object 
 7   joined_through_referral       36992 non-null  object 
 8   referral_id                   36992 non-null  object 
 9   preferred_offer_types         36704 non-null  object 
 10  medium_of_operation           36992 non-null  object 
 11  internet_option               36992 non-null  object 
 12  last_visit_time               36992 non-null  object 
 13  d

There are 7 numeric features and 17 categorical features in the dataset. The customer ID column (#0) is unnamed. Also, the counts of records are different for some features.

## 1.2 Target Variable (Class Imbalance)

## 1.4 Duplicates

## 1.5 Train-Validation-Test Split

## 1.6 Dropping Identification Features

## 1.7 Missing value

## 1.8 Data Patterns

## 1.9 Outliers

## 1.10 Feature Transformation

### 1.10.1 Categorical Encoding

### 1.10.2 Numerical Scaling

#Step 2: EDA

## 2.1 Target churn distribution

## 2.2 Descriptive Statistics

## 2.3 Data Distribution

## 2.4 Churn vs. Non-churn Comparison

## 2.5 Correlation Matrix

#Step 3: Feature Engineering

# Step 4: Feature Selection

## 4.1 Feature Reduction

## 4.2 Feature Importance (Using Random Forest)

#Step 5: Model Training

## 5.1 Logistic Regression

## 5.2 Naive Bayes

## 5.3 K nearest neighbors (kNN)

## 5.4 Random Forest

## 5.5 AdaBoost

## 5.6 XGBoost

#Step 6: Model Evaluation